In [44]:
import requests
import json
import re
import urllib.parse
from bs4 import BeautifulSoup
import time

# Название предмета
item_name = "Charm | Lil' Ava"
encoded_item_name = urllib.parse.quote(item_name)

# Заголовки и параметры
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                  ' Chrome/58.0.3029.110 Safari/537.36',
    'Referer': f'https://steamcommunity.com/market/listings/730/{encoded_item_name}',
}

# Базовые параметры запроса
params = {
    'query': '',
    'start': 0,
    'count': 100,  # Будем получать по 100 лотов
    'country': 'RU',
    'language': 'russian',  # Изменим на 'russian' для удобства
    'currency': '5',  # 5 - рубли
}

# Функция для получения общего количества лотов
def get_total_listings(url):
    response = requests.get(url, headers=headers)
    html = response.text
    # Парсим HTML с помощью BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # Ищем элемент с общим количеством лотов
    total_count_element = soup.find('span', {'id': 'searchResults_total'})
    if total_count_element:
        total_count = int(total_count_element.text.replace(',', ''))
        return total_count
    else:
        print('Не удалось определить общее количество лотов.')
        return 0

# Получаем общее количество лотов
base_url = f'https://steamcommunity.com/market/listings/730/{encoded_item_name}'
total_listings = get_total_listings(base_url)
if total_listings == 0:
    print('Нет доступных лотов для данного предмета.')
else:
    print(f'Общее количество лотов: {total_listings}')

    # Параметры для пагинации
    count_per_page = 100
    total_pages = (total_listings + count_per_page - 1) // count_per_page  # Округление вверх

    # Инициализируем список для хранения результатов
    results = []

    for page in range(total_pages):
        start = page * count_per_page
        # Обновляем параметры запроса
        params.update({
            'start': start,
            'count': count_per_page,
        })
        # Формируем URL для страницы с параметрами
        page_url = f'https://steamcommunity.com/market/listings/730/{encoded_item_name}/render/?query=&start={start}&count={count_per_page}&country=RU&language=english&currency=5'
  
        # Получаем данные страницы
        response = requests.get(page_url, headers=headers, params=params)

        # Проверяем статус код
        if response.status_code != 200:
            print(f'Ошибка при получении страницы {page + 1}. Статус код: {response.status_code}')
            continue

        # Попробуем получить JSON
        try:
            data = response.json()
        except json.JSONDecodeError as e:
            print(f'Ошибка при декодировании JSON на странице {page + 1}: {e}')
            print('Содержимое ответа:')
            print(response.text)
            continue  # Переходим к следующей странице

        # Проверяем, содержит ли ответ нужные данные
        if data is None:
            print(f'Получен пустой ответ на странице {page + 1}')
            continue

        # Получаем HTML-код результатов
        results_html = data.get('results_html', '')
        if not results_html:
            print(f'Нет результатов на странице {page + 1}')
            continue

        # Парсим HTML с помощью BeautifulSoup
        soup = BeautifulSoup(results_html, 'html.parser')

        # Ищем все элементы с классом 'market_listing_row'
        listings = soup.find_all('div', class_='market_listing_row')

        for listing in listings:
            # Извлекаем ID лота
            listing_id = listing.get('id', '').replace('listing_', '')

            # Извлекаем цену
            price_span = listing.find('span', class_='market_listing_price_with_fee')
            if price_span:
                price_text = price_span.get_text(strip=True)
                # Извлекаем числовое значение цены
                price_value = re.sub(r'[^\d,]', '', price_text).replace(',', '.')
                try:
                    price = float(price_value)
                except ValueError:
                    price = None
            else:
                price = None

            # Извлекаем Charm Template
            charm_template_div = listing.find('div', class_='market_listing_row_details_attribute')
            if charm_template_div:
                charm_text = charm_template_div.get_text(strip=True)
                match = re.search(r'Шаблон брелка:\s*(\d+)', charm_text)
                if match:
                    charm_template_value = int(match.group(1))
                else:
                    charm_template_value = None
            else:
                charm_template_value = None

            # Если есть все необходимые данные, добавляем в результаты
            if price is not None and charm_template_value is not None:
                results.append({
                    'listing_id': listing_id,
                    'price': price,
                    'charm_template': charm_template_value,
                })

        # Выводим прогресс
        print(f'Засканирована страница {page + 1}/{total_pages} с лотами {start + 1} - {start + len(listings)}')

        # Задержка между запросами
        time.sleep(2)  # Задержка в 1 секунду


Общее количество лотов: 1865
https://steamcommunity.com/market/listings/730/Charm%20%7C%20Lil%27%20Ava/render/?query=&start=0&count=100&country=RU&language=english&currency=5
Засканирована страница 1/19 с лотами 1 - 100
https://steamcommunity.com/market/listings/730/Charm%20%7C%20Lil%27%20Ava/render/?query=&start=100&count=100&country=RU&language=english&currency=5
Засканирована страница 2/19 с лотами 101 - 200
https://steamcommunity.com/market/listings/730/Charm%20%7C%20Lil%27%20Ava/render/?query=&start=200&count=100&country=RU&language=english&currency=5
Засканирована страница 3/19 с лотами 201 - 300
https://steamcommunity.com/market/listings/730/Charm%20%7C%20Lil%27%20Ava/render/?query=&start=300&count=100&country=RU&language=english&currency=5
Засканирована страница 4/19 с лотами 301 - 400
https://steamcommunity.com/market/listings/730/Charm%20%7C%20Lil%27%20Ava/render/?query=&start=400&count=100&country=RU&language=english&currency=5
Засканирована страница 5/19 с лотами 401 - 500


In [49]:
from tabulate import tabulate
while True:
        try:
            min_value = int(input("Введите минимальное значение Charm Template: "))
            max_value = int(input("Введите максимальное значение Charm Template: "))
            if min_value > max_value:
                print("Минимальное значение не может быть больше максимального. Попробуйте снова.")
                continue
            break
        except ValueError:
            print("Пожалуйста, введите корректные целые числа.")

# Фильтруем результаты по заданному диапазону Charm Template
filtered_results = [item for item in results if min_value <= item['charm_template'] <= max_value]

    # Сортируем результаты по паттерну (от меньшей к большей)
filtered_results.sort(key=lambda x: x['charm_template'])

    # Выводим результаты
table_data = []
for item in filtered_results:
    table_data.append([
        item['listing_id'],
        f"{item['price']:.2f} руб.",
        item['charm_template']
    ])

# Определяем заголовки столбцов
headers = ['Listing ID', 'Price', 'Charm Template Value']

# Выводим таблицу
print(tabulate(table_data, headers=headers, tablefmt='pretty'))

+---------------------+--------------+----------------------+
|     Listing ID      |    Price     | Charm Template Value |
+---------------------+--------------+----------------------+
| 5180923443941087160 |  85.88 руб.  |        10003         |
| 5189930643183314447 | 2000.00 руб. |        10072         |
| 6989118151240617266 | 130.90 руб.  |        10225         |
| 5177546378189757621 | 516.55 руб.  |        10283         |
| 5182049977761950521 |  74.75 руб.  |        10329         |
| 5168539178942450618 |  82.20 руб.  |        10377         |
| 5184301777649982880 |  68.14 руб.  |        10426         |
| 5170790978745895984 | 447.23 руб.  |        10611         |
| 5195560776688965345 |  67.92 руб.  |        10643         |
| 5185427043550997367 | 645.70 руб.  |        10756         |
| 5188804743283457873 | 112.35 руб.  |        11122         |
| 5184301777649993230 |  68.14 руб.  |        11526         |
| 5191056543119427678 |  71.09 руб.  |        11684         |
| 517867